In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install langchain-text-splitters

In [2]:
# import installed libraries
from dotenv import load_dotenv
import os
import json
from openai import AzureOpenAI
import re

# from semantic_kernel.text import split_markdown_paragraph
# chunking the text into paragraphs using markdown
from langchain_text_splitters import MarkdownHeaderTextSplitter

# load environment variables
load_dotenv(override=True)


True

In [3]:
# loading the markdown file
markdown_file = os.path.join(os.getcwd(), "data", "DRAFT_Acolad_2023 - English.md")

chunked_data = []

with open(markdown_file, 'r') as file:
    data = file.read()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )
    chunked_data = markdown_splitter.split_text(data)

    print(len(chunked_data))

14


### Initializing the Azure OpenAI Client

In [4]:
# define openai Client
aoai_api_key = os.getenv("AZURE_OPENAI_KEY")
aoai_api_endpoint =  os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    api_key=aoai_api_key,
    api_version="2024-08-01-preview",
    azure_endpoint=aoai_api_endpoint
)

In [15]:

translated = []

for chunk in chunked_data:
        

    systemPrompt = f"""
    Your Task is to translate the given text from English to Spanish:
    DO NOT RESPOND WITH ANYTHING ELSE EXCEPT THE TRANSLATION.
    MAKE SURE TO KEEP THE MARKDOWN FORMATTING, THAT INCLUDES HEADERS, BULLETS, TABLES, ETC BUT DO NOT BOTHER WRAPPING IT AS MARKDOWN.
    Task:
        Step 1: Translate the given text to English, ensuring that the meaning (semantics) of the original text is preserved as closely as possible.
        Step 2: Do not translate any text containing "trademark symbols" (™ or ®). Keep these parts as is.
        Below is the text for translation:
        ----------------------------------------

        {chunk}
    """

    conversaion = [
        {"role": "system", "content": systemPrompt},
        {"role": "user", "content": "give me the translation in Spanish"}
    ]

    # Send the conversation to the API
    response = client.chat.completions.create(
        model=aoai_api_deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
        messages=conversaion,
        temperature=0.0,
    )

    # Print the assistant's response
    responseText = response.choices[0].message.content
    translated.append(responseText)


In [16]:

print(len(translated))
# concat all the translated text and create a new markdown file
translated_md = "\n\n".join(translated) 


# save the responseText to a file with utf-8 encoding
output_file = os.path.join(os.getcwd(), "data", "DRAFT_Acolad_2023 - Spanish test.md")
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(translated_md)

14
